# 一、Dataset 基础API的使用
### tf.data.Dataset.from_tensor_slices
### repeat,batch,interleave,map,shuffle,list_files


# 二、Dataset 读取csv文件:用','分割，按行存储的文件
### tf.data.TextLineDataset,tf.io.decode_csv


# 三、Dataset 读取tfrecord文件
### tf.train.FloatList, tf.train.Int64List, tf.train.BytesList
### tf.train.Feature, tf.train.Features, tf.train.Example
### example.SerializeToString
### tf.io.ParseSingleExample
### tf.io.VarlenFeature, tf.io.FixedLenFeature
### tf.data.TFRecordDataset, tf.io.TFRecordOptions

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl,np,pd,sklearn,tf,keras:
    print(module.__name__,module.__version__)

tf.test.is_gpu_available()

2.0.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.17.4
pandas 0.25.3
sklearn 0.22
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


False

# 从内存中去构件数据集

### 使用numpy的数据

In [2]:
# 参数可以是普通的列表，numpy的array，或者是一个字典
dataset = tf.data.Dataset.from_tensor_slices(np.arange(10)) 
print(dataset)

<TensorSliceDataset shapes: (), types: tf.int64>


In [3]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [4]:
# 1. repeat epoch
# 2. get batch
dataset = dataset.repeat(3) # 返回一个新的dataset，将之前的数据重复了3次
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype

In [5]:
dataset = dataset.batch(7) # 返回一个新的dataset，每7个数一组
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


In [6]:
# interleave: 对现有的dataset的每个元素去进行处理，处理完会返回一个新的结果；然后interleave会把新的结果在合并返回一个新的dataset数据集
# case: 文件名dataset —> 读取每个文件中具体数据后的数据集

dataset2 = dataset.interleave(
    # map_fn :去做什么样的变化
    lambda v: tf.data.Dataset.from_tensor_slices(v), # 将dataset中的每个item进行对应的处理
    # cycle_length: 并行程度，并行的取处理dataset中的多少个元素
    cycle_length = 5,
    # block_length: 从map_fn变化的结果当中每次取多少个元素出来
    block_length = 6, 
)
for item in dataset2:
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype

### 使用元组

In [7]:
x = np.array([[1,2],[3,4],[5,6]])
y = np.array(['cat','dag','fox'])
dataset3 = tf.data.Dataset.from_tensor_slices((x,y))
print(dataset3)

for item_x,item_y in dataset3:
    print(item_x,"=====",item_y)
    print(item_x.numpy(),"=====",item_y.numpy(),'\n')

<TensorSliceDataset shapes: ((2,), ()), types: (tf.int64, tf.string)>
tf.Tensor([1 2], shape=(2,), dtype=int64) ===== tf.Tensor(b'cat', shape=(), dtype=string)
[1 2] ===== b'cat' 

tf.Tensor([3 4], shape=(2,), dtype=int64) ===== tf.Tensor(b'dag', shape=(), dtype=string)
[3 4] ===== b'dag' 

tf.Tensor([5 6], shape=(2,), dtype=int64) ===== tf.Tensor(b'fox', shape=(), dtype=string)
[5 6] ===== b'fox' 



### 使用字典

In [8]:
dataset4 = tf.data.Dataset.from_tensor_slices({"feature":x,
                                               "label":y})
for item in dataset4:
    print(item)
    print(item["feature"],"=====",item['label'])
    print(item["feature"].numpy(),"=====",item['label'].numpy(),'\n')

{'feature': <tf.Tensor: id=133, shape=(2,), dtype=int64, numpy=array([1, 2])>, 'label': <tf.Tensor: id=134, shape=(), dtype=string, numpy=b'cat'>}
tf.Tensor([1 2], shape=(2,), dtype=int64) ===== tf.Tensor(b'cat', shape=(), dtype=string)
[1 2] ===== b'cat' 

{'feature': <tf.Tensor: id=135, shape=(2,), dtype=int64, numpy=array([3, 4])>, 'label': <tf.Tensor: id=136, shape=(), dtype=string, numpy=b'dag'>}
tf.Tensor([3 4], shape=(2,), dtype=int64) ===== tf.Tensor(b'dag', shape=(), dtype=string)
[3 4] ===== b'dag' 

{'feature': <tf.Tensor: id=137, shape=(2,), dtype=int64, numpy=array([5, 6])>, 'label': <tf.Tensor: id=138, shape=(), dtype=string, numpy=b'fox'>}
tf.Tensor([5 6], shape=(2,), dtype=int64) ===== tf.Tensor(b'fox', shape=(), dtype=string)
[5 6] ===== b'fox' 

